---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap snapdeal products.

---

In [1]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

---

# **Snapdeal Scraper**

- In this section, we will use the following user defined function to extract products from snapdeal website.

---

In [3]:
def main(search, num_of_items):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")
    driver.get('https://www.snapdeal.com/')

    # Search the products
    driver.find_element_by_xpath('//*[@id="inputValEnter"]').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_xpath('//*[@id="inputValEnter"]').send_keys(search)
    driver.find_element_by_xpath('//*[@id="inputValEnter"]').send_keys(Keys.RETURN)

    time.sleep(5)

    current_scroll_position = 0
    new_height = 0.8
    sum = 0
    speed = 8
    data_items = num_of_items
    records = list()

    while sum <= data_items:

        # Screen scroller to get similar items
        while current_scroll_position <= new_height:

            current_scroll_position += speed
            driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(0.015)

        # Parse items from start to the scrolled location of the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('section', attrs={'class': 'dp-fired'})
        for result in results:

            items = result.findAll('div', {'class': 'product-tuple-description'})
            for item in items:
                try:
                    URL = item.find('a', {'class': 'dp-widget-link'})['href']
                except AttributeError:
                    URL = ''

                try:
                    title = item.find('p', {'class': 'product-title'}).text
                except AttributeError:
                    title = ''

                try:
                    striked_price = item.find('span', {'class': 'product-desc-price'}).text
                except AttributeError:
                    striked_price = ''

                try:
                    price = item.find('span', {'class': 'product-price'}).text
                except AttributeError:
                    price = ''

                try:
                    price_off = item.find('div', {'class': 'product-discount'}).text.strip()
                except AttributeError:
                    price_off = ''

                try:
                    ratings_count = item.find('p', {'class': 'product-rating-count'}).text.replace('(', '').replace(')', '')
                except AttributeError:
                    ratings_count = ''

                tuple_record = (URL, title, price, striked_price, price_off, ratings_count)
                records.append(tuple_record)

            sum = sum + len(items)

        time.sleep(2)
        
        if sum <= data_items:
            button = driver.find_element_by_xpath('//*[@id="see-more-products"]')
            driver.execute_script("arguments[0].click();", button)

    driver.close()

    # # Saving file to a CSV
    data = pd.DataFrame(data=records, columns=['URL', 'Title', 'Price', 'StrikedPrice', 'PriceOff', 'RatingsCount'])
    data.to_csv('results.csv', index=False)
    print('Records saved to the file!')

In [4]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, num_of_items=280)

Records saved to the file!
